In [1]:
import os
import sys
import numpy as np
import pandas as pd
import pickle
import tqdm
import matplotlib.pyplot as plt
import re

root = os.path.dirname(os.path.dirname(os.path.realpath('__file__')))
sys.path.insert(0, os.path.join(root, 'src'))

from models import models_dir

In [4]:
epochs = 1000
trained_models = os.listdir(f'{models_dir}TSOC')
trained_models = [trained_model for trained_model in trained_models if f'epochs={epochs}' in trained_model]
trained_models

['TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_ort=False_seedmat=0_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0.pth',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=50_opt=sgd_

In [5]:
def parse_parameters(model_name):
    model_name = model_name.split('.pth')[0]
    model_name = model_name.split('TSOC-')[1]
    params = model_name.split('_')
    param_dict = {}
    for param in params:
        key, value = param.split('=')
        # Attempt to convert to int, float, or keep as string
        if value.isdigit():
            param_dict[key] = int(value)
        else:
            try:
                param_dict[key] = float(value)
            except ValueError:
                # Special case for booleans
                if value.lower() in ['true', 'false']:
                    param_dict[key] = value.lower() == 'true'
                else:
                    param_dict[key] = value

    return param_dict

## TSOC reconstruction evaluation

### Evaluate models

In [6]:
for model_name in trained_models:
    params = parse_parameters(model_name)
    orthogonal_flag = "--orthogonal" if params['ort'] else ""
    heart_rate = params['hr'].replace('-', ' ')
    command = f"""
        python ../experiments/decoder_evaluation.py \
            --n {params['n']} \
            --m {params['m']} \
            --seed_training {params['seedtrain']} \
            --mode {params['mode']} \
            --seed_matrix {params['seedmat']} \
            --isnr {params['isnr']} \
            --N_train {params['N']} \
            --train_fraction {params['tf']} \
            --N_test 10000 \
            --basis sym6 \
            --fs {params['fs']} \
            --heart_rate {heart_rate} \
            {orthogonal_flag} \
            --source {params['src']} \
            --processes 48 \
            --gpu 3 \
            --min_lr {params['minlr']} \
            --min_delta {params['mind']} \
            --patience {params['p']} \
            --batch_size {params['bs']} \
            --threshold {params['th']} \
            --epochs {params['epochs']} \
            --lr {params['lr']} \
            --optimizer {params['opt']}
        """
    os.system(command)

INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=True, source=best, seed_matrix=52744, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=standard, orthogonal=False, source=random, seed_matrix=0, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


Model
TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_ort=False_seedmat=0_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0
has already been evaluated


INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=64, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=30808, seed_training=0, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=0.0001, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


...

100%|██████████| 10000/10000 [00:03<00:00, 2548.52it/s]
INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=128, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=88219, seed_training=1, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=1e-05, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


P: 35.038  TP: 29.318  TPR: 0.841  TNR: 0.988  ACC: 0.946


100%|██████████| 10000/10000 [00:03<00:00, 2523.80it/s]
INFO:root:Running test with parameters: n=128, m=48, epochs=1000, lr=0.001, opt=adam, batch_size=128, N_train=2000000, basis=sym6, fs=256, heart_rate=[60, 100], isnr=35, mode=rakeness, orthogonal=True, source=best, seed_matrix=77732, seed_training=1, processes=48, threshold=0.5, gpu=3, train_fraction=0.9, min_lr=1e-05, min_delta=1e-05, patience=40, N_test=10000, corr_name=96af96a7ddfcb2f6059092c250e18f2a, support_method=TSOC, seed_train_data=11, seed_test_data=66, M=1000, loc=0.25, results_folder=/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction, model_folder=/srv/newpenny/dnn-cs/tsoc/trained_models/TSOC


P: 35.018  TP: 29.305  TPR: 0.841  TNR: 0.988  ACC: 0.947


100%|██████████| 10000/10000 [00:03<00:00, 2730.38it/s]


### Aggregate performance results

In [7]:
results_folder = '/srv/newpenny/dnn-cs/tsoc/results/TSOC/reconstruction'
os.listdir(results_folder)


['TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=64_opt=adam_lr=0.001_th=0.5_tf=0.9_minlr=1e-05_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=best_ort=True_seedmat=52744_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=rakeness_src=best_ort=True_seedmat=30808_epochs=1000_bs=50_opt=sgd_lr=0.1_th=0.5_tf=0.9_minlr=0.001_p=40_mind=0.0001_seeddata=11_seedtrain=0_corr=96af96a7ddfcb2f6059092c250e18f2a_loc=0.25.pkl',
 'TSOC-N=2000000_n=128_m=48_fs=256_hr=60-100_isnr=35_mode=standard_src=random_or

In [27]:
m_list = [32, 48, 64]
mode_list = ['std', 'rak']
ort_list = [True, False]
seed_mat_list = []
seed_train_list = [0, 1, 2, 3, 4]
opt_list = ['adam', 'sgd']
bs_list = [32, 64, 128]
min_delta_list = [1e-4, 1e-5]

index = pd.MultiIndex.from_product(
    [m_list, mode_list, ort_list, seed_mat_list, seed_train_list, opt_list, bs_list, min_delta_list],
      names=['m', 'mode', 'ort', 'seedmat', 'seedtrain', 'opt', 'bs', 'mind'])

In [28]:
results = pd.DataFrame(index=index, columns=['P', 'TP', 'TPR', 'TNR', 'ACC', 'RSNR'], dtype=np.float64)
# results = pd.DataFrame(columns=['P', 'TP', 'TPR', 'TNR', 'ACC', 'RSNR'], dtype=np.float64)
for model_name in trained_models:
    params = parse_parameters(model_name)
    results_path = os.path.join(results_folder, f"{model_name.split('.pth')[0]}.pkl")
    if os.path.exists(results_path):
        result = pd.read_pickle(results_path)
        results.loc[params['m'], params['mode'], params['ort'], params['seedmat'], params['seedtrain'], params['opt'], params['bs'], params['mind']] = result

In [29]:
# min delta
results.xs(key=30808, level='seedmat').xs(key=64, level='bs').sort_values('RSNR', ascending=False)

P         TP       TPR  \
m  mode     ort  seedtrain opt  mind                                      
48 rakeness True 1         adam 0.00001  35.219447  29.312898  0.835748   
                 0         adam 0.00001  35.219447  29.344447  0.836768   
                 1         adam 0.00010  35.219447  29.294785  0.835287   
                 0         adam 0.00010  35.219447  29.273587  0.834667   

                                              TNR       ACC       RSNR  
m  mode     ort  seedtrain opt  mind                                    
48 rakeness True 1         adam 0.00001  0.987916  0.944900  32.546757  
                 0         adam 0.00001  0.987927  0.945150  32.546387  
                 1         adam 0.00010  0.987997  0.944823  32.532460  
                 0         adam 0.00010  0.987873  0.944559  32.521255

In [30]:
# batch size
results.xs(key=30808, level='seedmat').xs(key=1e-5, level='mind').sort_values('RSNR', ascending=False)

P         TP       TPR       TNR  \
m  mode     ort  seedtrain opt  bs                                              
48 rakeness True 0         adam 128  35.222409  29.461828  0.839948  0.988268   
                 1         adam 128  35.222409  29.463608  0.839995  0.988369   
                                64   35.219447  29.312898  0.835748  0.987916   
                 0         adam 64   35.219447  29.344447  0.836768  0.987927   
                                32   35.217951  29.196985  0.832486  0.987760   

                                          ACC       RSNR  
m  mode     ort  seedtrain opt  bs                        
48 rakeness True 0         adam 128  0.946287  32.628118  
                 1         adam 128  0.946376  32.621216  
                                64   0.944900  32.546757  
                 0         adam 64   0.945150  32.546387  
                                32   0.943901  32.473391

In [18]:
results.xs(key=128, level='bs').sort_values('RSNR', ascending=False)

P         TP  \
m  mode     ort  seedmat seedtrain opt  min_d                           
48 rakeness True 30808   0         adam 0.00001  35.222409  29.461828   
                         1         adam 0.00001  35.222409  29.463608   
                 77732   1         adam 0.00001  35.017900  29.305281   
                 88219   0         adam 0.00001  35.038370  29.324763   
                         1         adam 0.00001  35.038370  29.318335   
                 75680   0         adam 0.00001  35.109771  29.315862   
                 77732   0         adam 0.00001  35.017900  29.245945   
                 75680   1         adam 0.00001  35.109771  29.294205   

                                                      TPR       TNR       ACC  \
m  mode     ort  seedmat seedtrain opt  min_d                                   
48 rakeness True 30808   0         adam 0.00001  0.839948  0.988268  0.946287   
                         1         adam 0.00001  0.839995  0.988369  0.946376   
                 77732   1         adam 0.00001  0.840939  0.988395  0.946731   
                 88219   0         adam 0.00001  0.840735  0.987978  0.946424   
                         1         adam 0.00001  0.840520  0.987808  0.946247   
                 75680   0         adam 0.00001  0.838741  0.988267  0.946019   
                 77732   0         adam 0.00001  0.839264  0.988349  0.946241   
                 75680   1         adam 0.00001  0.838148  0.988275  0.945859   

                                                      RSNR  
m  mode     ort  seedmat seedtrain opt  min_d               
48 rakeness True 30808   0         adam 0.00001  32.628118  
                         1         adam 0.00001  32.621216  
                 77732   1         adam 0.00001  32.542928  
                 88219   0         adam 0.00001  32.530562  
                         1         adam 0.00001  32.521453  
                 75680   0         adam 0.00001  32.521272  
                 77732   0         adam 0.00001  32.510246  
                 75680   1         adam 0.00001  32.502708

In [ ]:

results.loc[48].sort_values('RSNR', ascending=False)

P         TP  \
mode     ort   seedmat seedtrain opt  bs  min_d                           
rakeness True  30808   0         adam 128 0.00001  35.222409  29.461828   
                       1         adam 128 0.00001  35.222409  29.463608   
                                      64  0.00001  35.219447  29.312898   
                       0         adam 64  0.00001  35.219447  29.344447   
               77732   1         adam 128 0.00001  35.017900  29.305281   
               30808   1         adam 64  0.00010  35.219447  29.294785   
               88219   0         adam 128 0.00001  35.038370  29.324763   
                       1         adam 128 0.00001  35.038370  29.318335   
               75680   0         adam 128 0.00001  35.109771  29.315862   
               30808   0         adam 64  0.00010  35.219447  29.273587   
               77732   0         adam 128 0.00001  35.017900  29.245945   
               75680   1         adam 128 0.00001  35.109771  29.294205   
               30808   0         adam 32  0.00001  35.217951  29.196985   
               88219   0         adam 64  0.00001  35.039809  29.206011   
               77732   0         adam 64  0.00001  35.013635  29.155454   
               75680   1         adam 64  0.00001  35.113654  29.215267   
                       0         adam 64  0.00001  35.113654  29.190983   
               30808   0         sgd  50  0.00010  35.217199  29.159999   
               75680   0         adam 32  0.00001  35.115615  29.102037   
standard True  52744   0         adam 64  0.00010  34.463575  28.784733   
                                 sgd  50  0.00010  34.466099  28.725199   
         False 0       0         sgd  50  0.00010  34.746199  28.941499   
                                 adam 64  0.00010  34.750995  28.851015   

                                                        TPR       TNR  \
mode     ort   seedmat seedtrain opt  bs  min_d                         
rakeness True  30808   0         adam 128 0.00001  0.839948  0.988268   
                       1         adam 128 0.00001  0.839995  0.988369   
                                      64  0.00001  0.835748  0.987916   
                       0         adam 64  0.00001  0.836768  0.987927   
               77732   1         adam 128 0.00001  0.840939  0.988395   
               30808   1         adam 64  0.00010  0.835287  0.987997   
               88219   0         adam 128 0.00001  0.840735  0.987978   
                       1         adam 128 0.00001  0.840520  0.987808   
               75680   0         adam 128 0.00001  0.838741  0.988267   
               30808   0         adam 64  0.00010  0.834667  0.987873   
               77732   0         adam 128 0.00001  0.839264  0.988349   
               75680   1         adam 128 0.00001  0.838148  0.988275   
               30808   0         adam 32  0.00001  0.832486  0.987760   
               88219   0         adam 64  0.00001  0.837228  0.987726   
               77732   0         adam 64  0.00001  0.836748  0.988049   
               75680   1         adam 64  0.00001  0.835760  0.988035   
                       0         adam 64  0.00001  0.835040  0.987892   
               30808   0         sgd  50  0.00010  0.831367  0.986779   
               75680   0         adam 32  0.00001  0.832422  0.987523   
standard True  52744   0         adam 64  0.00010  0.840723  0.987531   
                                 sgd  50  0.00010  0.838870  0.986719   
         False 0       0         sgd  50  0.00010  0.839155  0.986920   
                                 adam 64  0.00010  0.836358  0.986727   

                                                        ACC       RSNR  
mode     ort   seedmat seedtrain opt  bs  min_d                         
rakeness True  30808   0         adam 128 0.00001  0.946287  32.628118  
                       1         adam 128 0.00001  0.946376  32.621216  
                                      64  0.00001  0.944900  32.5

In [85]:
import torch
print(torch.__version__)

2.5.1
